<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-08 11:20:06
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.33 C
-------------------
Today PnL: -32.82 K (-0.25%)
Current PnL: -21.68 L (-14.75%)
CY Booked + Current PnL: -13.41 L (-9.12%)
-------------------
Total profit:  2.15 L
Total loss:  -23.83 L
-------------------
Total Booked + Current PnL: 13.57 L (11.3%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 79.94 L (60.02%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.12,H-SC,13.80,164279.0,38079.0,8099.0,6.72,30.17,4.93,36.59,156.0,4.70,1.27,46.47,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.67,108536.0,4224.0,12427.0,-0.51,4.05,11.45,15.96,10.0,0.34,0.84,21.63,X5K,ATH,METALS
67,SBIN,760.30,863.00,-12.46,M-LC,3.90,212305.0,10826.0,16390.0,-0.50,5.37,7.72,13.51,62.0,0.66,1.64,18.65,XY25,NTT,BANKS
71,SIS,477.00,477.00,2286.53,H-SC,7.16,64209.0,-16881.0,16881.0,1.67,-20.82,26.29,0.00,157.0,-1.00,0.50,30.13,OX40N,NTT,MISC
82,TTKPRESTIG,769.29,770.00,-9.35,H-SC,11.16,82949.0,-17828.0,17917.0,-0.13,-17.69,21.60,0.09,145.0,-1.00,0.64,8.98,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,PGHH,13388.00,18062.58,-20.74,H-MC,6.63,200100.0,-720.0,70835.0,0.93,-0.36,35.40,34.92,84.0,-0.01,1.54,4.30,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.73,99645.0,-859.0,31498.0,-0.92,-0.85,31.61,30.49,81.0,-0.03,0.77,31.27,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.94,204089.0,-411.0,41818.0,-0.09,-0.20,20.49,20.25,9.0,-0.01,1.57,8.51,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.15,H-LC,2.55,170442.0,-30944.0,118559.0,0.61,-15.37,69.56,43.51,19.0,-0.26,1.31,18.64,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.95,H-LC,3.53,180900.0,-19410.0,79379.0,-0.25,-9.69,43.88,29.94,24.0,-0.24,1.40,24.98,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.83,H-LC,3.64,184992.0,-15514.0,64285.0,-0.20,-7.74,34.75,24.33,30.0,-0.24,1.43,6.68,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-58.90,H-LC,3.88,130514.0,-134912.0,191816.0,-0.16,-50.83,146.97,21.44,26.0,-0.70,1.01,4.88,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,28.12,H-SC,13.80,164279.0,38079.0,8099.0,6.72,30.17,4.93,36.59,156.0,4.70,1.27,46.47,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.0,-51.71,L-SC,22.02,70426.0,-43123.0,83173.0,3.50,-37.98,118.10,35.27,267.0,-0.52,0.54,72.37,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.69,119286.0,-23193.0,23809.0,2.17,-16.28,19.96,0.44,163.0,-0.97,0.92,40.79,OX40N,NTT,BANKS
71,SIS,477.00,477.0,2286.53,H-SC,7.16,64209.0,-16881.0,16881.0,1.67,-20.82,26.29,0.00,157.0,-1.00,0.50,30.13,OX40N,NTT,MISC
62,REPCOHOME,515.07,880.0,-57.53,H-SC,3.87,156593.0,-48405.0,193643.0,1.50,-23.61,123.66,70.85,146.0,-0.25,1.21,26.41,XY24,NTT,FINANCE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LAOPALA,369.40,464.00,118.16,H-SC,3.47,69356.0,-31490.0,57316.0,-3.86,-31.23,82.64,25.61,141.0,-0.55,0.53,28.59,AR,NTT,CERAMICS
50,KPIGREEN,497.21,731.64,26.30,H-SC,13.34,129213.0,3916.0,55161.0,-3.63,3.13,42.69,47.15,144.0,0.07,1.00,61.65,MH,ATH,POWER
17,BSOFT,628.85,836.99,-22.90,H-SC,13.76,64724.0,-45325.0,81753.0,-3.17,-41.19,126.31,33.10,129.0,-0.55,0.50,7.55,XR,ATH,IT
80,TITAGARH,1097.23,1548.00,-10.86,H-SC,6.04,146169.0,-45846.0,124726.0,-2.19,-23.88,85.33,41.08,158.0,-0.37,1.13,23.71,XY24,NTT,ENGINEERING
15,BLUESTARCO,1646.70,2326.38,-4.66,H-SC,7.96,178880.0,14210.0,53753.0,-2.12,8.63,30.05,41.28,119.0,0.26,1.38,17.30,X40N,ATH,AC


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,125.45,H-SC,3.90,147820.0,-26739.0,26770.0,-1.49,-15.32,18.11,0.02,135.0,-1.00,1.14,28.18,OX40N,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.69,119286.0,-23193.0,23809.0,2.17,-16.28,19.96,0.44,163.0,-0.97,0.92,40.79,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-9.35,H-SC,11.16,82949.0,-17828.0,17917.0,-0.13,-17.69,21.60,0.09,145.0,-1.00,0.64,8.98,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.0,-22.52,H-MC,2.14,81354.0,-21298.0,21290.0,-0.92,-20.75,26.17,-0.00,101.0,-1.00,0.63,23.67,OX40N,NTT,IT
71,SIS,477.00,477.0,2286.53,H-SC,7.16,64209.0,-16881.0,16881.0,1.67,-20.82,26.29,0.00,157.0,-1.00,0.50,30.13,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.06,H-LC,15.37,237510.0,-58424.0,152339.0,-0.07,-19.74,64.14,31.73,1.0,-0.38,1.83,1.40,X40,BTT,IT
88,VBL,492.64,671.64,-11.22,H-LC,11.01,269646.0,4606.0,91707.0,-0.30,1.74,34.01,36.33,3.0,0.05,2.08,15.27,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.67,H-LC,9.51,254395.0,-7206.0,152841.0,-1.09,-2.75,60.08,55.67,4.0,-0.05,1.96,3.18,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.34,H-LC,9.41,198510.0,-18009.0,110908.0,-0.37,-8.32,55.87,42.91,5.0,-0.16,1.53,3.96,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.97,H-LC,9.91,179511.0,-11135.0,57641.0,-0.28,-5.84,32.11,24.40,8.0,-0.19,1.38,9.16,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,28.12,H-SC,13.8,164279.0,38079.0,8099.0,6.72,30.17,4.93,36.59,156.0,4.7,1.27,46.47,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.12,H-SC,13.80,164279.0,38079.0,8099.0,6.72,30.17,4.93,36.59,156.0,4.70,1.27,46.47,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.64,26.30,H-SC,13.34,129213.0,3916.0,55161.0,-3.63,3.13,42.69,47.15,144.0,0.07,1.00,61.65,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,0.95,H-MC,6.09,162526.0,11613.0,45686.0,-0.37,7.69,28.11,37.97,90.0,0.25,1.25,34.95,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-4.66,H-SC,7.96,178880.0,14210.0,53753.0,-2.12,8.63,30.05,41.28,119.0,0.26,1.38,17.30,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.47,H-SC,2.37,106193.0,4189.0,39737.0,-0.40,4.11,37.42,43.06,120.0,0.11,0.82,21.93,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,0.95,H-MC,6.09,162526.0,11613.0,45686.0,-0.37,7.69,28.11,37.97,90.0,0.25,1.25,34.95,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-47.70,H-SC,13.36,136040.0,12704.0,121062.0,0.26,10.30,88.99,108.45,118.0,0.10,1.05,34.96,AR,ATH,MISC
90,WHIRLPOOL,1219.98,2270.00,-42.22,M-SC,4.86,97118.0,5620.0,73130.0,0.02,6.14,75.30,86.07,211.0,0.08,0.75,40.49,XR,NTT,DURABLES
50,KPIGREEN,497.21,731.64,26.30,H-SC,13.34,129213.0,3916.0,55161.0,-3.63,3.13,42.69,47.15,144.0,0.07,1.00,61.65,MH,ATH,POWER
91,WIPRO,243.46,420.00,-12.85,M-LC,6.49,148986.0,-1959.0,111412.0,-0.83,-1.30,74.78,72.51,55.0,-0.02,1.15,4.65,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2286.53,H-SC,7.16,64209.0,-16881.0,16881.0,1.67,-20.82,26.29,0.00,157.0,-1.00,0.50,30.13,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.02,70426.0,-43123.0,83173.0,3.50,-37.98,118.10,35.27,267.0,-0.52,0.54,72.37,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.17,H-SC,17.90,89178.0,-11785.0,106318.0,-1.63,-11.67,119.22,93.63,149.0,-0.11,0.69,32.83,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,16.00,91020.0,-6960.0,28826.0,0.15,-7.10,31.67,22.32,151.0,-0.24,0.70,32.01,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.67,108536.0,4224.0,12427.0,-0.51,4.05,11.45,15.96,10.0,0.34,0.84,21.63,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.12,H-SC,13.80,164279.0,38079.0,8099.0,6.72,30.17,4.93,36.59,156.0,4.70,1.27,46.47,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.26,H-SC,8.08,218518.0,25498.0,22551.0,0.01,13.21,10.32,24.89,128.0,1.13,1.69,39.64,X40N,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,946.25,M-SC,8.15,142197.0,-38946.0,84408.0,-0.26,-21.50,59.36,25.10,233.0,-0.46,1.10,13.96,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,0.95,H-MC,6.09,162526.0,11613.0,45686.0,-0.37,7.69,28.11,37.97,90.0,0.25,1.25,34.95,AR,ATH,AUTO
88,VBL,492.64,671.64,-11.22,H-LC,11.01,269646.0,4606.0,91707.0,-0.30,1.74,34.01,36.33,3.0,0.05,2.08,15.27,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.42
1,25,41.26
2,50,71.32


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.07
LC    36.52
MC    22.42
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.25
X40N     15.41
X40      13.45
XY25     10.80
XR        9.06
AR        7.76
OX40N     7.72
X5K       2.26
MH        1.77
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.28
H-SC    27.67
H-MC    19.43
M-SC    11.94
M-LC     7.15
M-MC     2.64
L-SC     1.46
L-LC     1.09
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.39,-10.42,45.91
IT,12.05,-19.42,77.42
FINANCE,8.38,-19.93,63.44
BANKS,7.66,-14.83,61.68
PAINTS,5.80,-16.07,38.95
HEALTHCARE,5.27,3.71,20.60
MISC,5.20,-21.44,92.44
ELECTRICAL,4.78,-13.40,62.51
AUTO,4.44,-17.16,68.12


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2845942.0,23
XR,1118799.0,13
AR,1058001.0,9
X40,825469.0,10
XY25,648032.0,8
X40N,611117.0,12
OX40N,376587.0,10
SR,197205.0,2
X5K,146402.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2736163.0,30
H-LC,1485049.0,20
M-SC,1422628.0,15
H-MC,1187268.0,15
M-LC,451422.0,5
M-MC,335293.0,2
L-SC,272516.0,3
L-MC,57287.0,1
L-LC,45966.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      939983.0      7
M-SC       XY24      865581.0      7
H-SC       AR        619766.0      4
H-LC       X40       549974.0      6
H-SC       XR        524450.0      6
H-MC       XY24      469932.0      4
M-MC       XY24      335293.0      2
H-LC       X40N      315223.0      6
           XY25      250053.0      3
           AR        232485.0      2
H-MC       XY25      212752.0      2
H-SC       X40N      209715.0      4
H-MC       X40       199527.0      3
H-SC       SR        197205.0      2
           OX40N     189883.0      6
M-LC       XY24      189645.0      2
L-SC       XR        182620.0      2
M-SC       AR        160064.0      2
M-LC       X5K       133975.0      1
M-SC       XY25      122871.0      1
           XR        122626.0      2
H-MC       XR        120404.0      1
M-LC       XR        111412.0      1
L-SC       OX40N      89896.0      1
H-MC       X40N       86179.0      2
H-LC       X200       79379.0      1
M-SC       X40        75968.0      1
           OX40N      75518.0      2
L-MC       XR         57287.0      1
H-SC       MH         55161.0      1
L-LC       XY25       45966.0      1
H-MC       AR         45686.0      1
H-LC       XY24       45508.0      1
H-MC       MH         31498.0      1
           OX40N      21290.0      1
M-LC       XY25       16390.0      1
H-LC       X5K        12427.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
